In [1]:
# Standard library
import logging

import os
from pathlib import Path
import shlex
import time
import tqdm

# Third party
import hydra
import jsonlines
import rich

# Setup Logging
LOGGER = logging.getLogger(__name__)
logging.getLogger("common_retriever").setLevel(logging.WARNING)

# First Party
BASE_PATH = Path("/home/mila/g/gagnonju/DPR/")
CONF_PATH = BASE_PATH/"conf"
OUTPUTS = Path("/home/mila/g/gagnonju/IteratedDecoding/outputs/")
OUT_FILE = Path(
    "/home/mila/g/gagnonju/DPR/outputs/integrated_script_attempt/"
)
 
os.chdir(BASE_PATH)
import common_retriever

2021-09-09 22:58:28,939 [INFO] faiss.loader: Loading faiss with AVX2 support.
2021-09-09 22:58:28,940 [INFO] faiss.loader: Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2021-09-09 22:58:28,941 [INFO] faiss.loader: Loading faiss.
2021-09-09 22:58:29,148 [INFO] faiss.loader: Successfully loaded faiss.
[139633009272640] 2021-09-09 22:58:33,117 [INFO] transformers.file_utils: PyTorch version 1.7.0 available.


In [2]:
with hydra.initialize_config_dir(
    config_dir=str(CONF_PATH), 
):
    cfg = hydra.compose(
        config_name="dense_retriever",
        overrides=[
            f"out_file={shlex.quote(str(OUT_FILE))}",
            f"batch_size={1024}",
        ],
    )
    
    all_passages, id_prefixes = common_retriever.load_passages(cfg)
    n_docs = cfg.n_docs

/home/mila/g/gagnonju/.anaconda3/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'dense_retriever': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/home/mila/g/gagnonju/.anaconda3/lib/python3.8/site-packages/hydra/core/default_element.py:122: UserWarning: In 'ctx_sources/default_sources': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  deprecation_warning(
/home/mila/g/gagnonju/.anaconda3/lib/python3.8/site-packages/hydra/core/default_element.py:122: UserWarning: In 'datasets/retriever_default': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  deprecation_warning(
/home/mila/g/gagnonju/.anaconda3/lib/pyth

In [3]:
SETS = [
    "nq_train", 
    "nq_dev", 
    "nq_test",
]

with hydra.initialize_config_dir(
    config_dir=str(CONF_PATH), 
):
    cfg = hydra.compose(
        config_name="dense_retriever",
        overrides=[
            f"out_file={shlex.quote(str(OUT_FILE))}",
            f"batch_size={1024}",
        ],
    )
    
    datasets = {}
    for set_ in SETS:
        cfg = hydra.compose(
            config_name="dense_retriever",
            overrides=[
            f"out_file={shlex.quote(str(OUT_FILE))}",
            f"qa_dataset={set_}",
        ])
        datasets[set_] = common_retriever.load_data(cfg)

[139633009272640] 2021-09-09 23:02:38,604 [INFO] root: args.local_rank -1
[139633009272640] 2021-09-09 23:02:38,605 [INFO] root: WORLD_SIZE None
[139633009272640] 2021-09-09 23:02:38,606 [INFO] root: Initialized host cn-c005 as d.rank -1 on device=cuda, n_gpu=1, world size=1
[139633009272640] 2021-09-09 23:02:38,607 [INFO] root: 16-bits training: False 
[139633009272640] 2021-09-09 23:02:38,610 [INFO] dpr.data.download_data: Requested resource from https://dl.fbaipublicfiles.com/dpr/data/retriever/nq-train.qa.csv
[139633009272640] 2021-09-09 23:02:38,611 [INFO] dpr.data.download_data: Download root_dir /home/mila/g/gagnonju/DPR
[139633009272640] 2021-09-09 23:02:38,614 [INFO] dpr.data.download_data: File to be downloaded as /home/mila/g/gagnonju/DPR/downloads/data/retriever/qas/nq-train.csv
[139633009272640] 2021-09-09 23:02:38,615 [INFO] dpr.data.download_data: File already exist /home/mila/g/gagnonju/DPR/downloads/data/retriever/qas/nq-train.csv
[139633009272640] 2021-09-09 23:02:38,

In [4]:
print(id_prefixes)

['']


In [5]:
with hydra.initialize_config_dir(
    config_dir=str(CONF_PATH), 
):
    cfg = hydra.compose(
        config_name="dense_retriever",
        overrides=[
            f"out_file={shlex.quote(str(OUT_FILE))}",
            f"batch_size={1024}",
        ],
    )
    
    retriever = common_retriever.make_retriever(cfg, id_prefixes)

[139633009272640] 2021-09-09 23:02:40,116 [INFO] root: args.local_rank -1
[139633009272640] 2021-09-09 23:02:40,117 [INFO] root: WORLD_SIZE None
[139633009272640] 2021-09-09 23:02:40,118 [INFO] root: Initialized host cn-c005 as d.rank -1 on device=cuda, n_gpu=1, world size=1
[139633009272640] 2021-09-09 23:02:40,119 [INFO] root: 16-bits training: False 
[139633009272640] 2021-09-09 23:02:40,121 [INFO] root: Reading saved model from /home/mila/g/gagnonju/DPR/dpr/downloads/checkpoint/retriever/single/nq/bert-base-encoder.cp
[139633009272640] 2021-09-09 23:02:41,630 [INFO] root: model_state_dict keys odict_keys(['model_dict', 'optimizer_dict', 'scheduler_dict', 'offset', 'epoch', 'encoder_params'])
[139633009272640] 2021-09-09 23:02:41,895 [INFO] transformers.configuration_utils: loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/mila/g/gagnonju/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee2

In [12]:
SETS = [
    "nq_test",
    "nq_dev",
    "nq_train",
]

# This will take a very long time.
top_ids_and_scores = {}
questions_tensors = {}
assert hasattr(retriever, "batch_size")
retriever.batch_size = 1048 
n_docs = 100

# Size nq_test: 3610
# test, encoding: 17.62 sec with bs=1024
# test, encoding: 17.91 sec with bs=2048
# train, total 395.1392366751097 sec, bs=1024

for k in SETS:
    LOGGER.info(f"Set: {k}")
    LOGGER.info(f"{len(datasets[k].questions) = }")
    dataset = datasets[k]
    start = time.monotonic()
    questions_tensors[k] = retriever.generate_question_vectors(
        dataset.questions, 
        query_token=dataset.special_query_token,
    )
    delta = time.monotonic() - start
    
    LOGGER.info(f"{delta / len(dataset.questions)} tok/sec for batch size {retriever.batch_size}")
    LOGGER.info(f"Total time: {delta}")    

[139633009272640] 2021-09-09 23:15:03,718 [INFO] __main__: len(datasets[k]) = 3
[139633009272640] 2021-09-09 23:15:21,287 [INFO] root: Total encoded queries tensor torch.Size([3610, 768])
[139633009272640] 2021-09-09 23:15:21,293 [INFO] __main__: 205.4190261037647 tok/sec for batch size 1048
[139633009272640] 2021-09-09 23:15:21,293 [INFO] __main__: Total time: 17.573834656272084
[139633009272640] 2021-09-09 23:15:21,294 [INFO] __main__: len(datasets[k]) = 3
[139633009272640] 2021-09-09 23:16:04,273 [INFO] root: Total encoded queries tensor torch.Size([8757, 768])
[139633009272640] 2021-09-09 23:16:04,284 [INFO] __main__: 203.70176045644735 tok/sec for batch size 1048
[139633009272640] 2021-09-09 23:16:04,286 [INFO] __main__: Total time: 42.98931919084862
[139633009272640] 2021-09-09 23:16:04,287 [INFO] __main__: len(datasets[k]) = 3
[139633009272640] 2021-09-09 23:18:14,454 [INFO] root: Encoded queries 26200
[139633009272640] 2021-09-09 23:20:25,349 [INFO] root: Encoded queries 52400


In [13]:
for k in SETS:
    n_docs = 5
    # CPU:
    #   test, 310 sec for  5 docs
    #   train, 6530.344156157225 for 5 docs
    LOGGER.info(f"{questions_tensors[k].size(0) = }")
    start = time.monotonic()
    top_ids_and_scores[k] = retriever.get_top_docs(
        questions_tensors[k].numpy(), 
        n_docs,
    )
    delta = time.monotonic() - start
    LOGGER.info(f"{delta / len(dataset.questions)} sec/tok for n_docs {n_docs}")
    LOGGER.info(f"Total time: {delta}")  

[139633009272640] 2021-09-09 23:22:39,470 [INFO] __main__: questions_tensors[k].size(0) = 3610
[139633009272640] 2021-09-09 23:27:49,206 [INFO] root: index search time: 309.734111 sec.
[139633009272640] 2021-09-09 23:27:49,207 [INFO] __main__: 255.59851924341925 tok/sec for n_docs 5
[139633009272640] 2021-09-09 23:27:49,208 [INFO] __main__: Total time: 309.73575369035825
[139633009272640] 2021-09-09 23:27:49,208 [INFO] __main__: questions_tensors[k].size(0) = 8757
[139633009272640] 2021-09-09 23:39:13,167 [INFO] root: index search time: 683.957799 sec.
[139633009272640] 2021-09-09 23:39:13,168 [INFO] __main__: 115.74955812513724 tok/sec for n_docs 5
[139633009272640] 2021-09-09 23:39:13,169 [INFO] __main__: Total time: 683.9594144662842
[139633009272640] 2021-09-09 23:39:13,169 [INFO] __main__: questions_tensors[k].size(0) = 79168
[139633009272640] 2021-09-10 01:28:03,513 [INFO] root: index search time: 6530.342558 sec.
[139633009272640] 2021-09-10 01:28:03,514 [INFO] __main__: 12.1230

In [14]:
for k in SETS:
    output_target = OUTPUTS/f"nq_retrievals_{k}.jsonl"
    LOGGER.info(output_target)
    with jsonlines.open(output_target, "w") as fout:
        for question, answer, (indices, scores) in zip(            
            datasets[k].questions, 
            datasets[k].question_answers, 
            tqdm.tqdm(top_ids_and_scores[k])
        ):
            indices = [index for index in indices]
            f_scores = [float(score) for score in scores]
            fout.write(dict(
                question=question, 
                answer=answer,
                indices=indices, 
                scores=f_scores
            ))

[139633009272640] 2021-09-10 01:28:03,524 [INFO] __main__: /home/mila/g/gagnonju/IteratedDecoding/outputs/nq_retrievals_nq_test.jsonl
  0%|          | 0/3610 [00:00<?, ?it/s]
[139633009272640] 2021-09-10 01:28:03,585 [INFO] __main__: /home/mila/g/gagnonju/IteratedDecoding/outputs/nq_retrievals_nq_dev.jsonl
 87%|████████▋ | 7635/8757 [00:00<00:00, 64964.19it/s]
[139633009272640] 2021-09-10 01:28:03,707 [INFO] __main__: /home/mila/g/gagnonju/IteratedDecoding/outputs/nq_retrievals_nq_train.jsonl
100%|█████████▉| 78933/79168 [00:00<00:00, 87354.75it/s]
